Resampling: resample the audio to 4kHz.

Temporal Snippet Generation: The audio will be split into 5-second windows with a 50% overlap.

Baseline Removal Using DFT: We remove the baseline wandering (BW) component using DFT-based filtering.

Normalization: Normalize the snippets to the range [-1, 1].

In [4]:
import os

# 1. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 2. Define the starting path
drive_path = '/content/drive/MyDrive'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
/content/drive/MyDrive/Data Science Shared
/content/drive/MyDrive/Data Science AUEB
/content/drive/MyDrive/Data Science Shared/Text Analytics
/content/drive/MyDrive/Data Science Shared/Numerical Optimization 
/content/drive/MyDrive/Data Science Shared/Text Analytics/2nd Assignment
/content/drive/MyDrive/Data Science Shared/Text Analytics/1st Assignment
/content/drive/MyDrive/Data Science Shared/Text Analytics/3rd Assignment
/content/drive/MyDrive/Data Science Shared/Text Analytics/4th Assignment
/content/drive/MyDrive/Data Science Shared/Text Analytics/2nd Assignment/TA2025@MLPs
/content/drive/MyDrive/Data Science Shared/Text Analytics/2nd Assignment/TA2025@RNNS
/content/drive/MyDrive/Data Science Shared/Text Analytics/3rd Assignment/.ipynb_checkpoints
/content/drive/MyDrive/Data Science Shared/Text Analytics/4th Assignm

In [8]:
import os
import numpy as np
import scipy.io.wavfile as wav
from scipy.fft import fft, ifft

# Directory paths
base_dir = '/content/drive/MyDrive/Data Science AUEB/Deep Learning /datasets/'
initial_wav_dir = os.path.join(base_dir, 'initial wav files')
preprocessed_wav_dir = os.path.join(base_dir, 'preprocessed_initial_wav')

# Create the output directory if it doesn't exist
os.makedirs(preprocessed_wav_dir, exist_ok=True)

# Resample to 4kHz (if necessary)
def resample_signal(signal, original_fs, target_fs=4000):
    from scipy.signal import resample
    num_samples = int(len(signal) * float(target_fs) / original_fs)
    return resample(signal, num_samples)

# Temporal snippet generation
def generate_snippets(signal, window_size=5, overlap=0.5, fs=4000):
    snippet_length = window_size * fs
    step_size = int(snippet_length * (1 - overlap))
    snippets = []

    for start in range(0, len(signal) - snippet_length, step_size):
        snippet = signal[start:start + snippet_length]
        snippets.append(snippet)

    return snippets

# DFT-based baseline removal
def baseline_removal(snippet, fs=4000):
    N = len(snippet)
    # Compute the DFT of the snippet
    st = fft(snippet)

    # Frequency range for BW component (0-1 Hz)
    cutoff = int(np.ceil(1 * N / fs))

    # Zero out the frequencies lower than 1 Hz
    st[:cutoff] = 0
    st[-cutoff:] = 0

    # Perform inverse DFT
    cleaned_snippet = np.real(ifft(st))

    return cleaned_snippet

# Normalization
def normalize(snippet):
    return snippet / np.max(np.abs(snippet))

# Processing pipeline
def process_lung_sound(signal, fs=4000, window_size=5, overlap=0.5):
    # Resample to 4kHz
    resampled_signal = resample_signal(signal, fs, target_fs=fs)

    # Generate temporal snippets
    snippets = generate_snippets(resampled_signal, window_size, overlap, fs)

    # Process each snippet: remove baseline and normalize
    processed_snippets = []
    for snippet in snippets:
        snippet_bwf = baseline_removal(snippet, fs)
        snippet_normalized = normalize(snippet_bwf)
        processed_snippets.append(snippet_normalized)

    return processed_snippets

# Save the processed snippets as WAV files
def save_processed_snippets(snippets, output_dir, base_filename, fs=4000):
    for idx, snippet in enumerate(snippets):
        # Save each snippet as a separate file
        output_filename = os.path.join(output_dir, f"{base_filename}_snippet_{idx+1}.wav")
        wav.write(output_filename, fs, np.int16(snippet * 32767))  # Convert to 16-bit PCM

# Example usage with an audio file
def process_and_save_audio(file_path, output_dir):
    fs, signal = wav.read(file_path)

    # Process the signal
    processed_signal = process_lung_sound(signal, fs=fs)

    # Save the processed snippets
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    save_processed_snippets(processed_signal, output_dir, base_filename, fs)

# Process all WAV files in the initial_wav directory and save to preprocessed_wav_dir
for file_name in os.listdir(initial_wav_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(initial_wav_dir, file_name)
        process_and_save_audio(file_path, preprocessed_wav_dir)
